In [32]:
m

In [22]:
import geopandas as gpd
import folium
from folium.plugins import Fullscreen

# 1. Load shapefile and convert to WGS84
gdf = gpd.read_file("RTP_Linear.shp").to_crs(epsg=4326)

# 2. Base map
m = folium.Map(location=[39.0, -76.8], zoom_start=8, tiles="CartoDB Positron")
Fullscreen().add_to(m)

# 3. Color mapping
color_map = {
    "New Trail": "green",
    "Maintenance and Restoration": "orange",
    "Planning or Feasibility": "blue",
    "Trailhead or Amenity": "purple",
    "Bridge or Infrastructure": "red",
    "Unknown": "gray"
}

# 4. Plot trails
for _, row in gdf.iterrows():
    if row.geometry is None or row.geometry.is_empty:
        continue
    try:
        coords = [(lat, lon) for lon, lat, _ in row.geometry.coords]
    except:
        continue  # Skip multipart or invalid geometry

    project_type = row.get("PROJECT__1", "Unknown")
    color = color_map.get(project_type, "gray")
    name = row.get("PROJECT_NA", "Unnamed Trail")
    sponsor = row.get("SPONSOR", "Unknown")

    folium.PolyLine(
        locations=coords,
        color=color,
        weight=3,
        opacity=0.8,
        tooltip=f"{name} ({project_type})\nSponsor: {sponsor}"
    ).add_to(m)

# 5. Add a custom HTML legend
legend_html = """
<div style="
     position: fixed; 
     bottom: 30px; left: 30px; width: 250px; 
     z-index: 9999; font-size: 14px;
     background-color: white;
     border:2px solid grey;
     border-radius:6px;
     padding: 10px;
     box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
     ">
<b>Trail Project Type</b><br>
<span style="color:green;">&#9632;</span> New Trail<br>
<span style="color:orange;">&#9632;</span> Maintenance and Restoration<br>
<span style="color:blue;">&#9632;</span> Planning or Feasibility<br>
<span style="color:purple;">&#9632;</span> Trailhead or Amenity<br>
<span style="color:red;">&#9632;</span> Bridge or Infrastructure<br>
<span style="color:gray;">&#9632;</span> Unknown<br>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))



In [21]:
# Load and project the point shapefile
gdf_points = gpd.read_file("RTP_Point.shp").to_crs(epsg=4326)

# Add points to a separate FeatureGroup
point_layer = folium.FeatureGroup(name="Trail Project Points", show=True)

# Add each point to the FeatureGroup
for _, row in gdf_points.iterrows():
    if row.geometry is None or row.geometry.is_empty:
        continue

    # Get coordinates
    lat, lon = row.geometry.y, row.geometry.x

    # Popup info
    name = row.get("PROJECT_NA", "Unnamed Point")
    sponsor = row.get("SPONSOR", "Unknown")
    proj_type = row.get("PROJECT__1", "Unknown")

    location = row.get("PROJECT_LO")
    # Proper fallback for missing or string "None"
    if not location or str(location).strip().lower() == "none":
        location = f"Lat: {lat:.5f}, Lon: {lon:.5f}"

    fiscal_year = row.get("FISCAL_YEA", "N/A")
    district = row.get("DISTRICT", "Unknown")

    popup_text = f"""
    <b>{name}</b><br>
    <i>{proj_type}</i><br>
    <b>Sponsor:</b> {sponsor}<br>
    <b>District:</b> {district}<br>
    <b>Year:</b> {fiscal_year}<br>
    <b>Location:</b> {location}
    """

    # Add styled marker
    folium.CircleMarker(
        location=(lat, lon),
        radius=6,
        color='black',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.85,
        tooltip=name,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(point_layer)

# Add point layer to the map
point_layer.add_to(m)

# Add layer control to toggle trails vs points
folium.LayerControl().add_to(m)

In [28]:
# Available attributes
print(gdf.columns)
gdf.head()

print(gdf_points.columns)
gdf_points.head()

Index(['GlobalID', 'RT_REL_ID', 'RT_NUMBER', 'SPONSOR', 'PROJECT_NA',
       'PROJECT_LO', 'FISCAL_YEA', 'AWARD_AMOU', 'PROJECT_PH', 'RTP_MANAGE',
       'DISTRICT', 'COUNTY', 'PROJECT_DE', 'LEGISLATIV', 'COMPLETED_',
       'PROJECT_TY', 'MANAGED_US', 'PROJECT_CA', 'TRAIL_CLAS', 'LAND_WATER',
       'STRUC_REVI', 'OMT_CLEARA', 'MOU_EXECUT', 'MOU_EXTENS', 'NEPA_DATE',
       'ROW_CERTIF', 'INSPECTION', 'PUBLIC_DIS', 'CAPTURE_ME', 'OSM_ID',
       'OSM_REL_TY', 'PROJECT__1', 'FMIS_NUMBE', 'PROJECT_AU', 'AUTHORIZAT',
       'geometry'],
      dtype='object')
Index(['GlobalID', 'RT_NUMBER', 'SPONSOR', 'PROJECT_NA', 'PROJECT_LO',
       'FISCAL_YEA', 'AWARD_AMOU', 'PROJECT_PH', 'RTP_MANAGE', 'DISTRICT',
       'COUNTY', 'PROJECT_DE', 'LEGISLATIV', 'COMPLETED_', 'PROJECT_TY',
       'MANAGED_US', 'PROJECT_CA', 'TRAIL_CLAS', 'LAND_WATER', 'STRUC_REVI',
       'OMT_CLEARA', 'MOU_EXECUT', 'MOU_EXTENS', 'NEPA_DATE', 'ROW_CERTIF',
       'INSPECTION', 'PUBLIC_DIS', 'CAPTURE_ME', 'OSM_ID', 'OSM_R

,GlobalID,RT_NUMBER,SPONSOR,PROJECT_NA,PROJECT_LO,FISCAL_YEA,AWARD_AMOU,PROJECT_PH,RTP_MANAGE,DISTRICT,...,INSPECTION,PUBLIC_DIS,CAPTURE_ME,OSM_ID,OSM_REL_TY,PROJECT__1,FMIS_NUMBE,PROJECT_AU,AUTHORIZAT,geometry
0,e37c5e76-e98d-45eb-ad58-74f03ae01f0c,RT2201,Maryland Department of Transportation Port Adm...,Cox Creek Community Trail,"1000 Kembo Road, Curtis Bay, MD",2022,83491,Procurement,None,District 5,...,1899-12-30,Yes,None,0,None,New Trail,None,Yes,1899-12-30,POINT Z (1443617.079 557848.814 0)
1,779f3dfc-7ca8-4fd8-8d5c-176e32f6c923,RT2202,"Wicomico County Recreation, Parks & Tourism",Pirate's Wharf,"Whitehaven Rd, Quantico, MD",2022,50000,Procurement,None,District 1,...,1899-12-30,Yes,None,0,None,New Trail,None,No,1899-12-30,POINT Z (1671611.33 241068.432 0)
2,d848295b-3622-4373-ae4b-3b3b37f6979e,RT2203,Queen Anne’s County Parks and Recreation,Kent Narrows Trail,321 Piney Narrows Road,2022,59040,MOU Execution / NEPA,None,District 2,...,1899-12-30,Yes,None,0,None,New Trail,None,None,1899-12-30,POINT Z (1527491.593 475643.28 0)
3,60f85ad9-a8a7-476a-b036-26e3f298ba3e,RT2204,Mid-Atlantic Off Road Enthusiasts,Rivers Edge Trails,"13th Ave., Brunswick, MD",2022,35520,Construction / Invoicing,None,District 7,...,1899-12-30,Yes,None,0,None,New Trail,None,None,1899-12-30,POINT Z (1140645.649 600459.238 0)
4,3ed22eeb-1b37-42e2-adcf-f1ff2572c174,RT2206,Washington County Department of Public Works,Washington County Regional Park,None,2022,120000,ROW,None,District 6,...,1899-12-30,Yes,None,0,None,New Trail,None,None,1899-12-30,POINT Z (1123450.767 710020.581 0)


In [27]:
print(gdf['PROJECT_LO'].head(20))
print(gdf_points['PROJECT_LO'].head(20))

0               None
1               None
2               None
3               None
4               None
5               None
6               None
7               None
8               None
9               None
10    MTB Loop Trail
11              None
12    Howard Co side
13     MoCo Mullinix
14              None
15              None
16         Segment 2
17              None
18              None
19              None
Name: PROJECT_LO, dtype: object
0       1000 Kembo Road, Curtis Bay, MD
1           Whitehaven Rd, Quantico, MD
2                321 Piney Narrows Road
3              13th Ave., Brunswick, MD
4                                  None
5     Catoctin Furnace Rd, Thurmont, MD
6                                  None
7                                  None
8                                  None
9                                  None
10                                 None
11                                 None
12                                 None
13                          

In [29]:
# First 20 values of PROJECT_LO
print(gdf_points['PROJECT_LO'].head(20))

# Summary of unique values and counts
print(gdf_points['PROJECT_LO'].value_counts(dropna=False))

# Number of missing or null values
missing_count = gdf_points['PROJECT_LO'].isnull().sum()
print(f"Number of null/missing entries in PROJECT_LO: {missing_count}")

# Check for strings like "None", empty strings, or whitespace
none_like = gdf_points['PROJECT_LO'].apply(lambda x: str(x).strip().lower() in ['none', '', 'nan'])
print(f"Number of 'None'-like entries: {none_like.sum()}")

# Optionally view all unique non-null values
unique_non_null = gdf_points.loc[~gdf_points['PROJECT_LO'].isnull(), 'PROJECT_LO'].unique()
print(f"Sample unique non-null PROJECT_LO values:\n{unique_non_null[:10]}")

0       1000 Kembo Road, Curtis Bay, MD
1           Whitehaven Rd, Quantico, MD
2                321 Piney Narrows Road
3              13th Ave., Brunswick, MD
4                                  None
5     Catoctin Furnace Rd, Thurmont, MD
6                                  None
7                                  None
8                                  None
9                                  None
10                                 None
11                                 None
12                                 None
13                                 None
14                                 None
15                                 None
16                                 None
17                                 None
18                                 None
19                                 None
Name: PROJECT_LO, dtype: object
PROJECT_LO
None                                  129
13th Ave., Brunswick, MD                2
1000 Kembo Road, Curtis Bay, MD         1
321 Piney Narrows Road         

In [31]:
# First 20 values of PROJECT_LO
print(gdf['PROJECT_LO'].head(20))

# Summary of unique values and counts
print(gdf['PROJECT_LO'].value_counts(dropna=False))

# Number of missing or null values
missing_count = gdf['PROJECT_LO'].isnull().sum()
print(f"Number of null/missing entries in PROJECT_LO: {missing_count}")

# Check for strings like "None", empty strings, or whitespace
none_like = gdf['PROJECT_LO'].apply(lambda x: str(x).strip().lower() in ['none', '', 'nan'])
print(f"Number of 'None'-like entries: {none_like.sum()}")

# Optionally view all unique non-null values
unique_non_null = gdf.loc[~gdf['PROJECT_LO'].isnull(), 'PROJECT_LO'].unique()
print(f"Sample unique non-null PROJECT_LO values:\n{unique_non_null[:10]}")

0               None
1               None
2               None
3               None
4               None
5               None
6               None
7               None
8               None
9               None
10    MTB Loop Trail
11              None
12    Howard Co side
13     MoCo Mullinix
14              None
15              None
16         Segment 2
17              None
18              None
19              None
Name: PROJECT_LO, dtype: object
PROJECT_LO
None              48
MTB Loop Trail     1
Howard Co side     1
MoCo Mullinix      1
Segment 2          1
Hiker Biker        1
Black Marsh        1
Name: count, dtype: int64
Number of null/missing entries in PROJECT_LO: 48
Number of 'None'-like entries: 48
Sample unique non-null PROJECT_LO values:
['MTB Loop Trail' 'Howard Co side' 'MoCo Mullinix' 'Segment 2'
 'Hiker Biker' 'Black Marsh']
